In [1]:
from pathlib import Path
from functools import partial

import numpy as np
from plotly import graph_objs as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from ipywidgets import widgets
from damask_parse import read_table

# Functions

In [2]:
def read_non_uniform_csv(path, delimiter=',', skip_rows=0, header_row=1):
    'Load CSV file with variable length columns into a numpy array.'

    path = Path(path)
    
    arrs = []
    headers = None
    with path.open() as handle:        
        for ln_idx, ln in enumerate(handle):
            
            ln = ln.strip()

            if header_row is not None and ln_idx == header_row:
                headers = [i for i in ln.split(delimiter)]
            
            if ln_idx < skip_rows:
                continue

            ln_arr = []
            for i in ln.split(delimiter):
                try:
                    i_parse = float(i)
                except ValueError:
                    i_parse = np.nan
                ln_arr.append(i_parse)

            arrs.append(ln_arr)

    arrs = np.array(arrs)
    
    if headers:
        return headers, arrs
    else:
        return arrs

def read_tensile_test_data(path):
    'Read tensile test data CSV into a list of dict.'

    path = Path(path)
    delimiter = ','
    
    # Read data:
    headers, arr = read_non_uniform_csv(
        path, delimiter=delimiter, skip_rows=2, header_row=1)
    
    # Get orientations for each stress/strain column pair:
    with path.open() as handle:
        ln = handle.readline()    
        oris = [int(i.split()[0]) for i in ln.split(delimiter)[::2]]

    map_keys = {
        'eng. strain': 'eng_strain',
        'eng. stress /Mpa': 'eng_stress',
    }
    out = [
        {
            'orientation': ori,
            map_keys[headers[ori_idx]]: arr[:, ori_idx],
            map_keys[headers[ori_idx + 1]]: arr[:, ori_idx + 1],
        }
        for ori_idx, ori in enumerate(oris)
    ]
    
    return out

def get_true_stress_strain(eng_stress, eng_strain):
    'Convert engineering stress/strain to true stress/strain'
    
    common = 1 + eng_strain
    true_stress = eng_stress * (common)
    true_strain = np.log(common)
    
    return true_stress, true_strain

def get_eng_stress_strain(true_stress, true_strain):
    'Convert true stress/strain to engineering stress/strain.'
    eng_strain = np.exp(true_strain) - 1
    eng_stress = true_stress / (1 + eng_strain)
    
    return eng_stress, eng_strain
    
def find_nearest_index(arr, val):
    'Find the 1D array index whose value is closest to some value.'
    return np.nanargmin(np.abs(arr - val))

## Tests

In [3]:
def test_find_nearest_index():
    arr = np.array([1, 2, 3, 4, 5])
    val = 4.3
    assert find_nearest_index(arr, val) == 3
    
test_find_nearest_index()

# Classes

In [11]:
class HardeningLawFitter(object):
        
    DEFAULT_THETA_0 = 250 
    DEFAULT_TAU_SAT = 80
    FIG_WIDTH = 380
    FIG_HEIGHT = 280
    FIG_MARG = {
        't': 50,
        'l': 60,
        'r': 50,
        'b': 80,
    }
    FIG_PAD = [0.01, 5]
    STEP_SIZE = 1e-5
        
    def __init__(self, exp_stress, exp_strain, youngs_modulus=None, trial_taylor_factor=None,
                 hardening_law_params=None):
        
        exp_test = TensileTest(exp_stress, exp_strain,
                               youngs_modulus=youngs_modulus,
                               taylor_factor=trial_taylor_factor)
        
        if not hardening_law_params:
            hardening_law_params = {
                'theta_0': HardeningLawFitter.DEFAULT_THETA_0,
                'tau_sat': HardeningLawFitter.DEFAULT_TAU_SAT,
            }
                                    
        initial_values = [exp_test.shear_strain[0], exp_test.shear_stress[0]]
        hlaw = HardeningLaw(**hardening_law_params, initial_values=initial_values)
        hlaw.solve(exp_test.shear_strain[-1], HardeningLawFitter.STEP_SIZE)
        
        self.tensile_tests = [exp_test]            
        self.hardening_laws = [hlaw]
        self.taylor_factors = [exp_test.taylor_factor]
        
        self._widgets = self._generate_widgets()
        self._visual = None
        
    def add_simulated_tensile_test(self, true_stress, true_strain):
        
        sim_test = TensileTest(
            true_stress=stress/1e6,
            true_strain=strain,
            youngs_modulus=self.exp_tensile_test.youngs_modulus,
            plastic_range=self.exp_tensile_test.plastic_range,
        )
        self.tensile_tests.append(sim_test)        
        sim_idx = len(self.tensile_tests) - 2
        
        ss_type = self._widgets['macro_stress_strain']['controls']['stress_strain_type'].value
        if ss_type == 'Engineering':
            macro_stress = sim_test.eng_stress
            macro_strain = sim_test.eng_strain
        else:
            macro_stress = sim_test.true_stress
            macro_strain = sim_test.true_strain
                
        # Add macroscopic curve:
        fig = self._widgets['macro_stress_strain']['fig']
        fig.add_trace(
            go.Scatter(
                x=macro_strain,
                y=macro_stress,
                name='Simulated #{}'.format(sim_idx + 1),
                line=go.scatter.Line(color=DEFAULT_PLOTLY_COLORS[sim_idx + 1]),
            )
        )
        self._widgets['macro_stress_strain']['fig_trace_idx']['macro_stress_strain'].append(len(fig.data) - 1)
        
        # Add plastic curve:
        fig = self._widgets['plastic_stress_strain']['fig']
        fig.add_trace(
            go.Scatter(
                x=sim_test.plastic_strain,
                y=sim_test.plastic_stress,
                name='Simulated #{}'.format(sim_idx + 1),
                line=go.scatter.Line(color=DEFAULT_PLOTLY_COLORS[sim_idx + 1]),
            )
        )
        self._widgets['plastic_stress_strain']['fig_trace_idx']['plastic_stress_strain'].append(len(fig.data) - 1)
        
        # Make youngs modulus, trial taylor factor and plastic range "read-only" if not already.
        if len(self.tensile_tests) == 2:
            self._widgets['macro_stress_strain']['controls']['plastic_range'].disabled = True
            self._widgets['plastic_stress_strain']['controls']['youngs_modulus'].disabled = True
            self._widgets['shear_stress_strain']['controls']['taylor_factor'].disabled = True
            
        # Estimate new Taylor factor:
        taylor_factor_est = []
        
        fractions = np.arange(0.1, 1.1, 0.1)
        min_shear_strain = self.exp_tensile_test.min_shear_strain
        range_shear_strain = self.exp_tensile_test.range_shear_strain
        min_plastic_strain = sim_test.min_plastic_strain
        range_plastic_strain = sim_test.range_plastic_strain
                
        for i in fractions:
            
            shear_strain = min_shear_strain + i * range_shear_strain                        
            shear_idx = find_nearest_index(self.exp_tensile_test.shear_strain, shear_strain)
            shear_stress = self.exp_tensile_test.shear_stress[shear_idx]
            
            plastic_strain = min_plastic_strain + i * range_plastic_strain            
            plastic_idx = find_nearest_index(sim_test.plastic_strain, plastic_strain)
            plastic_stress = sim_test.plastic_stress[plastic_idx]
            
            taylor_factor_est.append(plastic_stress / shear_stress)

        new_taylor_factor = np.mean(taylor_factor_est)
        self.taylor_factors.append(new_taylor_factor)
        
        # Add shear stress/strain curve using new taylor factor:
        new_sstress, new_sstrain = self.exp_tensile_test.get_shear_stress_strain(new_taylor_factor)
        fig = self._widgets['shear_stress_strain']['fig']
        fig.add_trace(
            go.Scatter(
                x=new_sstrain,
                y=new_sstress,
                name='M<sub>{}</sub> = {:.2f}'.format(sim_idx + 1, new_taylor_factor),
                line=go.scatter.Line(color=DEFAULT_PLOTLY_COLORS[sim_idx + 1]),
            )
        )
        self._widgets['shear_stress_strain']['fig_trace_idx']['shear_stress_strain'].append(len(fig.data) - 1)
        
        # Add a new HardeningLaw, with starting parameters the same as previous:
        hardening_law_params = {
            'theta_0': self.hardening_laws[-1].theta_0,
            'tau_sat': self.hardening_laws[-1].tau_sat,
        }
                                    
        initial_values = [new_sstrain[0], new_sstress[0]]
        hlaw = HardeningLaw(**hardening_law_params, initial_values=initial_values)
        hlaw.solve(new_sstrain[-1], HardeningLawFitter.STEP_SIZE)        
        self.hardening_laws.append(hlaw)
        
        # Add new hardening law to widgets:
        fig.add_trace(
            go.Scatter(
                x=hlaw.gamma,
                y=hlaw.tau,
                name='Hard. law #{}'.format(sim_idx + 1),
                line=go.scatter.Line(color=DEFAULT_PLOTLY_COLORS[sim_idx + 1], dash='dash', width=1),
            )
        )
        self._widgets['shear_stress_strain']['fig_trace_idx']['hardening_law'].append(len(fig.data) - 1)
 
        fig = self._widgets['hardening_rate']['fig']
        fig.add_trace(
            go.Scatter(
                x=hlaw.gamma,
                y=hlaw.theta,
                name='Hard. law #{}'.format(sim_idx + 1),
                line=go.scatter.Line(color=DEFAULT_PLOTLY_COLORS[sim_idx + 1], dash='dash', width=1),
            )
        )
        self._widgets['hardening_rate']['fig_trace_idx']['hardening_rate'].append(len(fig.data) - 1)
        
        
    @property
    def trial_taylor_factor(self):
        return self.taylor_factors[0]
    
    @property
    def exp_tensile_test(self):
        return self.tensile_tests[0]
    
    def _update_widgets_stress_strain_type(self, change):
        
        ss_type = self._widgets['macro_stress_strain']['controls']['stress_strain_type']
        fig = self._widgets['macro_stress_strain']['fig']
        ss_trace_idx = self._widgets['macro_stress_strain']['fig_trace_idx']['macro_stress_strain']
        
        with fig.batch_update():
            # Update all macroscopic stress strain curves:
            for idx, i in enumerate(ss_trace_idx):
                if ss_type.value == 'Engineering':
                    stress = self.tensile_tests[idx].eng_stress
                    strain = self.tensile_tests[idx].eng_strain
                else:
                    stress = self.tensile_tests[idx].true_stress
                    strain = self.tensile_tests[idx].true_strain
                fig.data[i].x = strain
                fig.data[i].y = stress
            fig.layout.xaxis.title.text = ss_type.value + ' strain, ε'
            fig.layout.yaxis.title.text = ss_type.value + ' stress, σ / MPa'             
    
    def _update_widgets_plastic_range(self, change):
        
        # Only update first one, because if once multiple stress-strain curves, plastic range is fixed.
        
        plastic_range = self._widgets['macro_stress_strain']['controls']['plastic_range'].value
        self.exp_tensile_test.plastic_range = plastic_range

        fig_macro = self._widgets['macro_stress_strain']['fig']
        plastic_range_trace_idx = self._widgets['macro_stress_strain']['fig_trace_idx']['plastic_range_boundaries'][0]                
        with fig_macro.batch_update():            
            fig_macro.data[plastic_range_trace_idx].x = [
                self.exp_tensile_test.plastic_range[0]] * 2 + [None] + [
                self.exp_tensile_test.plastic_range[1]] * 2
            
            fig_macro.data[plastic_range_trace_idx].y = [
                -HardeningLawFitter.FIG_PAD[1], 
                HardeningLawFitter.FIG_PAD[1] + self.exp_tensile_test.max_stress,
                None,
                -HardeningLawFitter.FIG_PAD[1],
                HardeningLawFitter.FIG_PAD[1] + self.exp_tensile_test.max_stress,
            ]

        fig_plastic = self._widgets['plastic_stress_strain']['fig']
        stress_strain_trace_idx = self._widgets['plastic_stress_strain']['fig_trace_idx']['plastic_stress_strain'][0]
        with fig_plastic.batch_update():            
            fig_plastic.data[stress_strain_trace_idx].x = self.exp_tensile_test.plastic_strain
            fig_plastic.data[stress_strain_trace_idx].y = self.exp_tensile_test.plastic_stress
            fig_plastic.layout['xaxis']['range'] = [self.exp_tensile_test.min_plastic_strain,
                                                    self.exp_tensile_test.max_plastic_strain]
            fig_plastic.layout['yaxis']['range'] = [self.exp_tensile_test.min_plastic_stress,
                                                    self.exp_tensile_test.max_plastic_stress]

        fig_shear = self._widgets['shear_stress_strain']['fig']
        shear_trace_idx = self._widgets['shear_stress_strain']['fig_trace_idx']['shear_stress_strain'][0]
        self.exp_tensile_test._set_shear_stress_strain()        
        with fig_shear.batch_update():
            fig_shear.data[shear_trace_idx].x = self.exp_tensile_test.shear_strain
            fig_shear.data[shear_trace_idx].y = self.exp_tensile_test.shear_stress
            fig_shear.layout['xaxis']['range'] = [self.exp_tensile_test.min_shear_strain,
                                                  self.exp_tensile_test.max_shear_strain]
            fig_shear.layout['yaxis']['range'] = [self.exp_tensile_test.min_shear_stress,
                                                  self.exp_tensile_test.max_shear_stress]          
 
    def _update_trial_taylor_factor(self, change):
        
        # Trial taylor factor can only be changed when there is one tensile test so
        # only need to update first trace.
        
        tay_fac = self._widgets['shear_stress_strain']['controls']['taylor_factor'].value
        self.taylor_factors[0] = tay_fac
        self.exp_tensile_test.taylor_factor = tay_fac
        self.exp_tensile_test._set_shear_stress_strain()
        
        fig = self._widgets['shear_stress_strain']['fig']
        stress_strain_trace_idx = self._widgets['shear_stress_strain']['fig_trace_idx']['shear_stress_strain'][0]
        with fig.batch_update():
            fig.data[stress_strain_trace_idx].x = self.exp_tensile_test.shear_strain
            fig.data[stress_strain_trace_idx].y = self.exp_tensile_test.shear_stress
            fig.layout['xaxis']['range'] = [self.exp_tensile_test.min_shear_strain,
                                            self.exp_tensile_test.max_shear_strain]
            fig.layout['yaxis']['range'] = [self.exp_tensile_test.min_shear_stress,
                                            self.exp_tensile_test.max_shear_stress]
        
        hlaw = self.hardening_laws[-1]
        hlaw.solve(self.tensile_tests[-1].shear_strain[-1], HardeningLawFitter.STEP_SIZE)
        
        fig_shear = self._widgets['shear_stress_strain']['fig']
        shear_trace_idx = self._widgets['shear_stress_strain']['fig_trace_idx']['hardening_law'][0]
        with fig_shear.batch_update():
            fig_shear.data[shear_trace_idx].x = hlaw.gamma
            fig_shear.data[shear_trace_idx].y = hlaw.tau
        
        fig_hard = self._widgets['hardening_rate']['fig']
        hard_trace_idx = self._widgets['hardening_rate']['fig_trace_idx']['hardening_rate'][0]
        with fig_hard.batch_update():
            fig_hard.data[hard_trace_idx].x = hlaw.gamma
            fig_hard.data[hard_trace_idx].y = hlaw.theta        
                        
    def _update_youngs_modulus(self, change):
        
        # Young's modulus can only be changed when there is one tensile test so
        # only need to update first trace.        
        
        youngs_mod = self._widgets['plastic_stress_strain']['controls']['youngs_modulus'].value
        self.exp_tensile_test.youngs_modulus = youngs_mod
        self.exp_tensile_test._set_plastic_stress_strain()
        self.exp_tensile_test._set_shear_stress_strain()
        
        fig_plastic = self._widgets['plastic_stress_strain']['fig']
        stress_strain_trace_idx = self._widgets['plastic_stress_strain']['fig_trace_idx']['plastic_stress_strain'][0]
        with fig_plastic.batch_update():            
            fig_plastic.data[stress_strain_trace_idx].x = self.exp_tensile_test.plastic_strain
            fig_plastic.data[stress_strain_trace_idx].y = self.exp_tensile_test.plastic_stress
            fig_plastic.layout['xaxis']['range'] = [self.exp_tensile_test.min_plastic_strain,
                                                    self.exp_tensile_test.max_plastic_strain]
            fig_plastic.layout['yaxis']['range'] = [self.exp_tensile_test.min_plastic_stress,
                                                    self.exp_tensile_test.max_plastic_stress]

        fig_shear = self._widgets['shear_stress_strain']['fig']
        shear_trace_idx = self._widgets['shear_stress_strain']['fig_trace_idx']['shear_stress_strain'][0]
        self.exp_tensile_test._set_shear_stress_strain()        
        with fig_shear.batch_update():
            fig_shear.data[shear_trace_idx].x = self.exp_tensile_test.shear_strain
            fig_shear.data[shear_trace_idx].y = self.exp_tensile_test.shear_stress
            fig_shear.layout['xaxis']['range'] = [self.exp_tensile_test.min_shear_strain,
                                                  self.exp_tensile_test.max_shear_strain]
            fig_shear.layout['yaxis']['range'] = [self.exp_tensile_test.min_shear_stress,
                                                  self.exp_tensile_test.max_shear_stress]        
            
    def _update_hardening_rules(self, change):
        
        # Customisable hardening parameters should always refer to the final tensile test.
        
        theta_0 = self._widgets['hardening_rate']['controls']['hardening_law_theta_0'].value
        tau_sat = self._widgets['hardening_rate']['controls']['hardening_law_tau_sat'].value
        
        hlaw = self.hardening_laws[-1]
        hlaw.theta_0 = theta_0
        hlaw.tau_sat = tau_sat
        hlaw.solve(self.tensile_tests[-1].shear_strain[-1], HardeningLawFitter.STEP_SIZE)
        
        fig_shear = self._widgets['shear_stress_strain']['fig']
        shear_trace_idx = self._widgets['shear_stress_strain']['fig_trace_idx']['hardening_law'][-1]
        with fig_shear.batch_update():
            fig_shear.data[shear_trace_idx].x = hlaw.gamma
            fig_shear.data[shear_trace_idx].y = hlaw.tau
        
        fig_hard = self._widgets['hardening_rate']['fig']
        hard_trace_idx = self._widgets['hardening_rate']['fig_trace_idx']['hardening_rate'][-1]
        with fig_hard.batch_update():
            fig_hard.data[hard_trace_idx].x = hlaw.gamma
            fig_hard.data[hard_trace_idx].y = hlaw.theta        
            
    def _generate_widgets_macro_stress_strain(self):
        
        data = [
            {
                'x': self.exp_tensile_test.eng_strain,
                'y': self.exp_tensile_test.eng_stress,
                'name': 'Experimental',
                'line': {
                    'color': DEFAULT_PLOTLY_COLORS[0],
                },
            },
            {
                'x': [self.exp_tensile_test.plastic_range[0]] * 2 + [None] + [
                      self.exp_tensile_test.plastic_range[1]] * 2,
                'y': [
                    -HardeningLawFitter.FIG_PAD[1], 
                    HardeningLawFitter.FIG_PAD[1] + self.exp_tensile_test.max_stress,
                    None,
                    -HardeningLawFitter.FIG_PAD[1],
                    HardeningLawFitter.FIG_PAD[1] + self.exp_tensile_test.max_stress,
                ],
                'mode': 'lines',
                'line': {
                    'color': '#888',
                    'width': 2,
                },
                'showlegend': False,
            }            
        ]
        layout = {
            'title': 'Experimental Data',
            'width': HardeningLawFitter.FIG_WIDTH,
            'height': HardeningLawFitter.FIG_HEIGHT,
            'margin': HardeningLawFitter.FIG_MARG,
            'xaxis': {
                'title': 'Engineering strain, ε',
                'range': [-HardeningLawFitter.FIG_PAD[0],
                          HardeningLawFitter.FIG_PAD[0] + self.exp_tensile_test.max_strain],
            },
            'yaxis': {
                'title': 'Engineering stress, σ / MPa',
                'range': [-HardeningLawFitter.FIG_PAD[1],
                          HardeningLawFitter.FIG_PAD[1] + self.exp_tensile_test.max_stress],
            },            
        }

        widget_ss_type = widgets.RadioButtons(
            options=['Engineering', 'True'],
            description='Stress/strain:',
            value='Engineering',
        )                
        plastic_range_widget = widgets.FloatRangeSlider(
            value=self.exp_tensile_test.plastic_range,
            step=0.005,
            min=self.exp_tensile_test.min_true_strain,
            max=self.exp_tensile_test.max_true_strain,
            description='Plastic range:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout_format='.4f',
            layout=widgets.Layout(width='90%'),
        )
        widget_ss_type.observe(self._update_widgets_stress_strain_type, names='value')        
        plastic_range_widget.observe(self._update_widgets_plastic_range, names='value')        
        out = {
            'fig': go.FigureWidget(data=data, layout=layout),
            'fig_trace_idx': {
                'macro_stress_strain': [0],
                'plastic_range_boundaries': [1],
            },
            'controls': {
                'stress_strain_type': widget_ss_type,
                'plastic_range': plastic_range_widget,
            },
        }
        
        return out
    
    def _generate_widgets_plastic_stress_strain(self):
              
        data = [
            {
                'x': self.exp_tensile_test.plastic_strain,
                'y': self.exp_tensile_test.plastic_stress,
                'name': 'Experimental',
                'line': {
                    'color': DEFAULT_PLOTLY_COLORS[0],
                },                
            },        
        ]
        layout = {
            'title': 'Plastic strain',
            'width': HardeningLawFitter.FIG_WIDTH,
            'height': HardeningLawFitter.FIG_HEIGHT,
            'margin': HardeningLawFitter.FIG_MARG,
            'xaxis': {
                'title': 'Plastic strain, εₚ',
                'range': [self.exp_tensile_test.min_plastic_strain,
                          self.exp_tensile_test.max_plastic_strain],
            },
            'yaxis': {
                'title': 'True stress, σ / MPa',
                'range': [self.exp_tensile_test.min_plastic_stress,
                          self.exp_tensile_test.max_plastic_stress],
            },
            'showlegend': True,
        }
        
        youngs_mod_widget = widgets.BoundedFloatText(
            value=self.exp_tensile_test.youngs_modulus,
            description='Young\'s modulus (GPa):',                    
            min=1,
            max=2000,
            step=0.1,
            style={'description_width': 'initial'},
        )
        youngs_mod_widget.observe(self._update_youngs_modulus, names='value')
        
        out = {
            'fig': go.FigureWidget(data=data, layout=layout),
            'fig_trace_idx': {
                'plastic_stress_strain': [0],
            },
            'controls': {
                'youngs_modulus': youngs_mod_widget,
            },
        }

        return out        
    
    def _generate_widgets_shear_stress_strain(self):
        
        hlaw = self.hardening_laws[0]        
        data = [
            {
                'x': self.exp_tensile_test.shear_strain,
                'y': self.exp_tensile_test.shear_stress,
                'name': 'M<sub>0</sub> = {:.2f}'.format(self.trial_taylor_factor),
                'line': {
                    'color': DEFAULT_PLOTLY_COLORS[0],
                },                
            },
            {
                'x': hlaw.gamma,
                'y': hlaw.tau,                          
                'name': 'Hard. law #0',
                'line': {
                    'color': DEFAULT_PLOTLY_COLORS[0],
                    'dash': 'dash',
                    'width': 1,
                },
            }            
        ]                   
        layout = {
            'title': 'Single crystal',
            'width': HardeningLawFitter.FIG_WIDTH,
            'height': HardeningLawFitter.FIG_HEIGHT,
            'margin': HardeningLawFitter.FIG_MARG,
            'xaxis': {
                'title': 'Shear strain, γ',
                'range': [self.exp_tensile_test.min_shear_strain,
                          self.exp_tensile_test.max_shear_strain],
            },
            'yaxis': {
                'title': 'Shear stress, τ / MPa',
                'range': [self.exp_tensile_test.min_shear_stress,
                          self.exp_tensile_test.max_shear_stress],
            },
        }
        
        taylor_factor_widget = widgets.BoundedFloatText(
            value=self.exp_tensile_test.taylor_factor,
            description=r'Trial taylor factor, $M_{0}$:',
            min=1,
            max=5,
            step=0.1,
            style={'description_width': 'initial'},
        )                        
        taylor_factor_widget.observe(self._update_trial_taylor_factor, names='value')
        
        out = {
            'fig': go.FigureWidget(data=data, layout=layout),
            'fig_trace_idx': {
                'shear_stress_strain': [0],
                'hardening_law': [1],        
            },
            'controls': {
                'taylor_factor': taylor_factor_widget,
            },
        }
        
        return out
        
    def _generate_widgets_hardening_rate(self):
        
        hlaw = self.hardening_laws[0]        
        data = [
            {
                'x': hlaw.gamma,
                'y': hlaw.theta,
                'name': 'Hard. law #0',
                'line': {
                    'color': DEFAULT_PLOTLY_COLORS[0],
                    'dash': 'dash',
                    'width': 1,
                },
            },
        ]                   
        layout = {
            'title': 'Hardening rate',
            'width': HardeningLawFitter.FIG_WIDTH,
            'height': HardeningLawFitter.FIG_HEIGHT,
            'margin': HardeningLawFitter.FIG_MARG,
            'xaxis': {
                'title': 'Shear strain, γ',
                'range': [self.exp_tensile_test.min_shear_strain,
                          self.exp_tensile_test.max_shear_strain],
            },
            'yaxis': {
                'title': 'Hardening rate, θ',
            },
            'showlegend': True,
        }
    
        hardening_label_widget = widgets.HBox([
            widgets.Label(value='Hardening law: '),
            widgets.Label(value=r'$\theta = \theta_{0}\left(1 - \tau/\tau_{\textrm{sat}}\right)$')
        ])

        hardening_theta_0_widget = widgets.FloatText(
            value=hlaw.theta_0,
            description=r'$\theta_{0}$ (MPa)',
        )
        hardening_tau_sat_widget = widgets.FloatText(
            value=hlaw.tau_sat,
            description=r'$\tau_{\textrm{sat}}$ (MPa)',
        )
        hardening_theta_0_widget.observe(self._update_hardening_rules, names='value')
        hardening_tau_sat_widget.observe(self._update_hardening_rules, names='value')
        
        out = {
            'fig': go.FigureWidget(data=data, layout=layout),
            'fig_trace_idx': {
                'hardening_rate': [0],       
            },
            'controls': {
                'hardening_law_label': hardening_label_widget,
                
                'hardening_law_theta_0': hardening_theta_0_widget,
                'hardening_law_tau_sat': hardening_tau_sat_widget,
            },
        }
        
        return out
    
    def _generate_widgets(self):
    
        out = {
            'macro_stress_strain': self._generate_widgets_macro_stress_strain(),            
            'plastic_stress_strain': self._generate_widgets_plastic_stress_strain(),
            'shear_stress_strain': self._generate_widgets_shear_stress_strain(),
            'hardening_rate': self._generate_widgets_hardening_rate(),
        }
        
        return out   
    
    def _generate_visual(self):
        'Layout widgets.'
        
        sorted_widgets = []
        for i in [
            'macro_stress_strain',
            'plastic_stress_strain',
            'shear_stress_strain',
            'hardening_rate',
        ]:
            if i in self._widgets:
                sorted_widgets.append(self._widgets[i])
        
        vertical_children = []
        for w in sorted_widgets:
            horizontal_children = [w['fig']]
            vertical_sub_children = [v for k, v in w['controls'].items()]
            horizontal_children.append(
                widgets.VBox(
                    vertical_sub_children,
                    layout=widgets.Layout(
                        margin='5rem 0 0 5rem',
                        width='40%',
                        #border='1px solid red',
                    )
                )
            )
            
            vertical_children.append(
                widgets.HBox(
                    horizontal_children,
                    layout=widgets.Layout(
                        margin='0',
                        #border='1px solid blue',
                    )
                )
            )
            
        return widgets.VBox(vertical_children)        
    
    @property
    def visual(self):
        if not self._visual:
            self._visual = self._generate_visual()
        return self._visual
    
    def show(self, stress_strain_type='engineering'):
        'Plot stress/strain data'
        ss_type = {
            'engineering': 'Engineering',
            'true': 'True',
        }
        self._widgets['macro_stress_strain']['controls']['stress_strain_type'].value = ss_type[stress_strain_type]
        return self.visual
    
class HardeningLaw(object):
    
    def __init__(self, theta_0, tau_sat, initial_values, name='Untitled'):
        
        self.theta_0 = theta_0
        self.tau_sat = tau_sat
        self.initial_values = initial_values
        self.name = name
        
        self._gamma = None
        self._tau = None
        self._theta = None
        
    def hardening_rate(self, tau):
        theta = self.theta_0 * (1 - (tau / self.tau_sat))
        return theta
    
    def solve(self, final_strain, step_size):
        
        num_steps = int((final_strain - self.initial_values[0]) / step_size) + 1
        gamma_i = np.linspace(self.initial_values[0], final_strain, num_steps)
        tau_i = np.zeros_like(gamma_i) * np.nan
        theta_i = np.zeros_like(gamma_i) * np.nan
        
        tau_i[0] = self.initial_values[1]

        for j in range(num_steps - 1):
            theta_i[j] = self.hardening_rate(tau_i[j])
            tau_i[j + 1] = tau_i[j] + (step_size * theta_i[j])

        self._gamma = gamma_i
        self._tau = tau_i
        self._theta = theta_i
        
    @property
    def gamma(self):
        return self._gamma
    
    @property
    def tau(self):
        return self._tau
    
    @property
    def theta(self):
        return self._theta
        
class TensileTest(object):
    
    DEFAULT_TAYLOR_FACTOR = 2.5
    DEFAULT_YOUNGS_MOD = 79 # GPa
    DEFAULT_PLASTIC_RANGE_START = 0.02
    
    def __init__(self, eng_stress=None, eng_strain=None, true_stress=None, true_strain=None, 
                 youngs_modulus=None, taylor_factor=None, plastic_range=None):
        
        msg = 'Specify (`eng_stress` and `eng_strain`) or (`true_stress` and `true_strain`)'
        if eng_stress is None and eng_strain is None:
            if true_strain is None or true_stress is None:
                raise ValueError(msg)
            if len(true_strain) != len(true_strain):
                raise ValueError('Stress and strain do not have the same length.')
                
        if true_stress is None and true_strain is None:
            if eng_stress is None or eng_strain is None:
                raise ValueError(msg)
            if len(eng_stress) != len(eng_strain):
                raise ValueError('Stress and strain do not have the same length.')
        
        self._eng_stress = eng_stress
        self._eng_strain = eng_strain
        
        self._true_stress = true_stress
        self._true_strain = true_strain
        
        self.youngs_modulus = youngs_modulus or TensileTest.DEFAULT_YOUNGS_MOD
        self.taylor_factor = taylor_factor or TensileTest.DEFAULT_TAYLOR_FACTOR
        
        self.plastic_range = plastic_range        
        self._plastic_range_idx = None
        self._plastic_stress = None
        self._plastic_strain = None
        
        self._shear_stress = None
        self._shear_strain = None
            
    def __repr__(self):        
        out = '{}()'.format(self.__class__.__name__)
        return out
        
    def __len__(self):
        return len(self.eng_stress)
        
    def _set_true_stress_strain(self):
        'Convert engineering stress/strain to true stress/strain.'
        tstress, tstrain = get_true_stress_strain(self.eng_stress, self.eng_strain)
        self._true_stress = tstress
        self._true_strain = tstrain
        
    def _set_eng_stress_strain(self):
        'Convert true stress/strain to engineering stress/strain.'
        estress, estrain = get_eng_stress_strain(self.true_stress, self.true_strain)
        self._eng_stress = estress
        self._eng_strain = estrain
        
    def _set_plastic_stress_strain(self):
        'Use `plastic_range` to set plastic stress/strain.'

        idx = [find_nearest_index(self.true_strain, self.plastic_range[i]) for i in [0, 1]]
                
        stress_sub = self.true_stress[slice(*idx)]
        strain_sub = self.true_strain[slice(*idx)]        
        
        elastic_strain = stress_sub / self.youngs_modulus_MPa       
        self._plastic_strain = strain_sub - elastic_strain
        self._plastic_stress = stress_sub
        self._plastic_range_idx = idx
        
    def _set_shear_stress_strain(self):
        sstress, sstrain = self.get_shear_stress_strain(self.taylor_factor)        
        self._shear_strain = sstrain
        self._shear_stress = sstress
        
    @property
    def youngs_modulus_MPa(self):
        return self.youngs_modulus * 1e3
    
    @property
    def youngs_modulus_SI(self):
        return self.youngs_modulus * 1e9
        
    @property
    def plastic_range(self):
        return self._plastic_range
    
    @plastic_range.setter
    def plastic_range(self, plastic_range):
        # Validate within min-max and start<stop, set default otherwise.
        if not plastic_range:
            plastic_range = [TensileTest.DEFAULT_PLASTIC_RANGE_START,
                             self.true_strain[int(0.8 * len(self.true_strain))]]
            
        msg = ('Plastic range must be a range between the minimum and maximum '
               'true strain values: {} {}'.format(self.min_true_strain, self.max_true_strain))
        if plastic_range[0] < self.min_true_strain:
            raise ValueError(msg)
        if plastic_range[1] > self.max_true_strain:
            raise ValueError(msg)
            
        self._plastic_range = plastic_range
        self._set_plastic_stress_strain()
        self._set_shear_stress_strain()
        
    @property
    def eng_stress(self):
        if self._eng_stress is None:
            self._set_eng_stress_strain()
        return self._eng_stress
        
    @property
    def eng_strain(self):
        if self._eng_strain is None:
            self._set_eng_stress_strain()        
        return self._eng_strain        
        
    @property
    def true_stress(self):
        if self._true_stress is None:
            self._set_true_stress_strain()
        return self._true_stress
        
    @property
    def true_strain(self):
        if self._true_strain is None:
            self._set_true_stress_strain()
        return self._true_strain
    
    @property
    def max_stress(self):
        return np.nanmax([self.max_eng_stress, self.max_true_stress])
    
    @property
    def min_stress(self):
        return np.nanmin([self.min_eng_stress, self.min_true_stress])
    
    @property
    def max_true_stress(self):
        return np.nanmax(self.true_stress)
    
    @property
    def min_true_stress(self):
        return np.nanmin(self.true_stress)
    
    @property
    def max_eng_stress(self):
        return np.nanmax(self.eng_stress)
    
    @property
    def min_eng_stress(self):
        return np.nanmin(self.eng_stress)
    
    @property
    def max_strain(self):
        return np.nanmax([self.max_eng_strain, self.max_true_strain])
    
    @property
    def min_strain(self):
        return np.nanmin([self.min_eng_strain, self.min_true_strain])
            
    @property
    def max_true_strain(self):
        return np.nanmax(self.true_strain)
    
    @property
    def min_true_strain(self):
        return np.nanmin(self.true_strain)
    
    @property
    def max_eng_strain(self):
        return np.nanmax(self.eng_strain)
    
    @property
    def min_eng_strain(self):
        return np.nanmin(self.eng_strain)    
    
    @property
    def plastic_strain(self):
        if self._plastic_strain is None:
            self._set_plastic_stress_strain()
        return self._plastic_strain
    
    @property
    def plastic_stress(self):
        if self._plastic_stress is None:
            self._set_plastic_stress_strain()
        return self._plastic_stress
    
    @property
    def max_plastic_strain(self):
        return np.nanmax(self.plastic_strain)
    
    @property
    def min_plastic_strain(self):
        return np.nanmin(self.plastic_strain)
    
    @property
    def range_plastic_strain(self):
        return self.max_plastic_strain - self.min_plastic_strain
            
    @property
    def max_plastic_stress(self):
        return np.nanmax(self.plastic_stress)
    
    @property
    def min_plastic_stress(self):
        return np.nanmin(self.plastic_stress)
        
    @property
    def shear_stress(self):
        if self._shear_stress is None:
            self._set_shear_stress_strain()
        return self._shear_stress
    
    @property
    def shear_strain(self):
        if self._shear_strain is None:
            self._set_shear_stress_strain()
        return self._shear_strain    
        
    @property
    def max_shear_strain(self):
        return np.nanmax(self.shear_strain)
    
    @property
    def min_shear_strain(self):
        return np.nanmin(self.shear_strain)
    
    @property
    def range_shear_strain(self):
        return self.max_shear_strain - self.min_shear_strain
    
    @property
    def max_shear_stress(self):
        return np.nanmax(self.shear_stress)
    
    @property
    def min_shear_stress(self):
        return np.nanmin(self.shear_stress)        
    
    def get_shear_stress_strain(self, taylor_factor):
        'Use a Taylor factor to estimate single crystal shear strain/stress.'
        shear_strain = self.plastic_strain * taylor_factor
        shear_stress = self.plastic_stress / taylor_factor
        
        return shear_stress, shear_strain
    
pass

# Surfalex Data

In [5]:
data_path = Path(r'C:\Users\adamj\OneDrive - UOM\surfalex_tensile_test_data.csv')
data = read_tensile_test_data(data_path)

idx = 0
eng_stress = data[idx]['eng_stress']
eng_strain = data[idx]['eng_strain']

In [12]:
fitter = HardeningLawFitter(eng_stress, eng_strain, trial_taylor_factor=2.5)
fitter.show()

    'data': [{'line': {'color': 'rgb(31, 119, 180)'},
           …

This seems to fit OK-ish:
- theta_0: 400
- tau_sat: 95

Then:
- theta_0: 500
- tau_sat: 140

In [13]:
strain = table_data[0]['Mises(ln(V))']
stress = table_data[0]['Mises(Cauchy)']
fitter.add_simulated_tensile_test(true_stress=stress, true_strain=strain)

In [14]:
strain = table_data[1]['Mises(ln(V))']
stress = table_data[1]['Mises(Cauchy)']
fitter.add_simulated_tensile_test(true_stress=stress, true_strain=strain)

# Reading true stress-strain from Damask

In [7]:
base_path = Path(r'C:\Users\adamj\Dropbox (The University of Manchester)\sims_db\cp-fitting')
table_paths = [
    base_path.joinpath(r'2019-08-28-2331_70653\sims\0\postProc\geom_load.txt'), # 2000 grains 64^3, params 1    
#     base_path.joinpath(r'2019-08-30-0003_68388\sims\0\postProc\geom_load.txt'), # 500 grains 32^3, params 2
    base_path.joinpath(r'2019-08-30-0133_50864\sims\0\postProc\geom_load.txt'), # 2000 grains 32^3, params 2

]
table_data = [read_table(i) for i in table_paths]

In [8]:
f = go.FigureWidget(
    data=[
        {
            'x': i['Mises(ln(V))'],
            'y': i['Mises(Cauchy)'],
            'mode': 'lines+markers',
            'text': i['inc'],
        } for i in table_data
    ]
)
f

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'text': array([  0.,   1.,   2., ..., 258.…